In [1]:
#r "nuget: Newtonsoft.Json"
#r "nuget: Microsoft.Azure.Cosmos"
using Microsoft.Azure.Cosmos;
using System.Net.Http;

Installed Packages Microsoft.Azure.Cosmos, 3.53.1 Newtonsoft.Json, 13.0.4

Azure Cosmos DB Emulator

GatewayModeMaxConnectionLimit

ConnectionMode

AllowBulkExecution

In [7]:
var endpoint = "https://localhost:8081";
var key ="C2y6yDjf5/R+ob0N8A7Cgv30VRDJIWEHLM+4QDU5DE2nQ9nDuVTqobD4b8mGGyPMbIZnqyMsEcaGQy67XIw/Jw==";
var client = new CosmosClient(endpoint, key);

client.Display();

Microsoft.Azure.Cosmos.CosmosClient ClientOptions Microsoft.Azure.Cosmos.CosmosClientOptions ApplicationName <null> ApplicationRegion <null> ApplicationPreferredRegions <null> AccountInitializationCustomEndpoints <null> GatewayModeMaxConnectionLimit 50 RequestTimeout 00:00:06 TokenCredentialBackgroundRefreshInterval <null> CustomHandlers (empty) ConnectionMode Direct ConsistencyLevel <null> PriorityLevel <null> MaxRetryAttemptsOnRateLimitedRequests <null> MaxRetryWaitTimeOnRateLimitedRequests <null> EnableContentResponseOnWrite <null> UseSystemTextJsonSerializerWithOptions <null> IdleTcpConnectionTimeout <null> OpenTcpConnectionTimeout <null> MaxRequestsPerTcpConnection <null> MaxTcpConnectionsPerEndpoint <null> PortReuseMode <null> WebProxy <null> SerializerOptions <null> Serializer Microsoft.Azure.Cosmos.CosmosJsonSerializerWrapper LimitToEndpoint False AllowBulkExecution False EnableTcpConnectionEndpointRediscovery True HttpClientFactory <null> AvailabilityStrategy <null> ServerCertificateCustomValidationCallback <null> CosmosClientTelemetryOptions Microsoft.Azure.Cosmos.CosmosClientTelemetryOptions DisableSendingMetricsToService True DisableDistributedTracing True CosmosThresholdOptions Microsoft.Azure.Cosmos.CosmosThresholdOptions NonPointOperationLatencyThreshold 00:00:03 PointOperationLatencyThreshold 00:00:01 RequestChargeThreshold <null> PayloadSizeThresholdInBytes <null> QueryTextMode None FaultInjector <null> ResponseFactory Microsoft.Azure.Cosmos.CosmosResponseFactoryCore Endpoint https://localhost:8081/ AbsolutePath / AbsoluteUri https://localhost:8081/ LocalPath / Authority localhost:8081 HostNameType Dns IsDefaultPort False IsFile False IsLoopback True PathAndQuery / Segments [ / ] IsUnc False Host localhost Port 8081 Query Fragment Scheme https OriginalString https://localhost:8081 DnsSafeHost localhost IdnHost localhost IsAbsoluteUri True UserEscaped False UserInfo

In [12]:
CosmosClientOptions options = new CosmosClientOptions
{
    AllowBulkExecution = false,
    ApplicationName = "CosmosDBDemo",
    ApplicationRegion = "East US", 
    AvailabilityStrategy = AvailabilityStrategy.CrossRegionHedgingStrategy(threshold: TimeSpan.FromMilliseconds(500), thresholdStep: TimeSpan.FromMilliseconds(100)),
    ConsistencyLevel = ConsistencyLevel.Session,
    EnableContentResponseOnWrite = false,    
    RequestTimeout = TimeSpan.FromSeconds(30),    
    ConnectionMode = ConnectionMode.Gateway,
    MaxRetryAttemptsOnRateLimitedRequests = 5,
    MaxRetryWaitTimeOnRateLimitedRequests = TimeSpan.FromSeconds(30)    
};

client = new CosmosClient(endpoint, key, options);
client.Display();

Microsoft.Azure.Cosmos.CosmosClient ClientOptions Microsoft.Azure.Cosmos.CosmosClientOptions ApplicationName CosmosDBDemo ApplicationRegion East US ApplicationPreferredRegions <null> AccountInitializationCustomEndpoints <null> GatewayModeMaxConnectionLimit 50 RequestTimeout 00:00:30 TokenCredentialBackgroundRefreshInterval <null> CustomHandlers (empty) ConnectionMode Gateway ConsistencyLevel Session PriorityLevel <null> MaxRetryAttemptsOnRateLimitedRequests 5 MaxRetryWaitTimeOnRateLimitedRequests 00:00:30 EnableContentResponseOnWrite False UseSystemTextJsonSerializerWithOptions <null> IdleTcpConnectionTimeout <null> OpenTcpConnectionTimeout <null> MaxRequestsPerTcpConnection <null> MaxTcpConnectionsPerEndpoint <null> PortReuseMode <null> WebProxy <null> SerializerOptions <null> Serializer Microsoft.Azure.Cosmos.CosmosJsonSerializerWrapper LimitToEndpoint False AllowBulkExecution False EnableTcpConnectionEndpointRediscovery True HttpClientFactory <null> AvailabilityStrategy Microsoft.Azure.Cosmos.CrossRegionHedgingAvailabilityStrategy Threshold 00:00:00.5000000 ThresholdStep 00:00:00.1000000 EnableMultiWriteRegionHedge False IsSDKDefaultStrategyForPPAF False ServerCertificateCustomValidationCallback <null> CosmosClientTelemetryOptions Microsoft.Azure.Cosmos.CosmosClientTelemetryOptions DisableSendingMetricsToService True DisableDistributedTracing True CosmosThresholdOptions Microsoft.Azure.Cosmos.CosmosThresholdOptions NonPointOperationLatencyThreshold 00:00:03 PointOperationLatencyThreshold 00:00:01 RequestChargeThreshold <null> PayloadSizeThresholdInBytes <null> QueryTextMode None FaultInjector <null> ResponseFactory Microsoft.Azure.Cosmos.CosmosResponseFactoryCore Endpoint https://localhost:8081/ AbsolutePath / AbsoluteUri https://localhost:8081/ LocalPath / Authority localhost:8081 HostNameType Dns IsDefaultPort False IsFile False IsLoopback True PathAndQuery / Segments [ / ] IsUnc False Host localhost Port 8081 Query Fragment Scheme https OriginalString https://localhost:8081 DnsSafeHost localhost IdnHost localhost IsAbsoluteUri True UserEscaped False UserInfo

Meta data queries

In [8]:
using (var iterator = client.GetDatabaseQueryIterator<DatabaseProperties>())
{
    while (iterator.HasMoreResults)
    {
        var response = await iterator.ReadNextAsync();
        foreach (var db in response)
        {
            Console.WriteLine($"Database Id: {db.Id}");
        }
    }
}

Database Id: Live
Database Id: dbname
Database Id: StackOverflow


In [9]:
var livedb = await client.CreateDatabaseIfNotExistsAsync("Live");
livedb.Display();

Microsoft.Azure.Cosmos.DatabaseResponse Database Microsoft.Azure.Cosmos.DatabaseInlineCore Id Live Client Microsoft.Azure.Cosmos.CosmosClient ClientOptions Microsoft.Azure.Cosmos.CosmosClientOptions ApplicationName <null> ApplicationRegion <null> ApplicationPreferredRegions <null> AccountInitializationCustomEndpoints <null> GatewayModeMaxConnectionLimit 50 RequestTimeout 00:00:06 TokenCredentialBackgroundRefreshInterval <null> CustomHandlers (empty) ConnectionMode Direct ConsistencyLevel <null> PriorityLevel <null> MaxRetryAttemptsOnRateLimitedRequests <null> MaxRetryWaitTimeOnRateLimitedRequests <null> EnableContentResponseOnWrite <null> UseSystemTextJsonSerializerWithOptions <null> IdleTcpConnectionTimeout <null> OpenTcpConnectionTimeout <null> MaxRequestsPerTcpConnection <null> MaxTcpConnectionsPerEndpoint <null> PortReuseMode <null> WebProxy <null> SerializerOptions <null> Serializer Microsoft.Azure.Cosmos.CosmosJsonSerializerWrapper LimitToEndpoint False AllowBulkExecution False EnableTcpConnectionEndpointRediscovery True HttpClientFactory <null> AvailabilityStrategy <null> ServerCertificateCustomValidationCallback <null> CosmosClientTelemetryOptions Microsoft.Azure.Cosmos.CosmosClientTelemetryOptions DisableSendingMetricsToService True DisableDistributedTracing True CosmosThresholdOptions Microsoft.Azure.Cosmos.CosmosThresholdOptions NonPointOperationLatencyThreshold 00:00:03 PointOperationLatencyThreshold 00:00:01 RequestChargeThreshold <null> PayloadSizeThresholdInBytes <null> QueryTextMode None FaultInjector <null> ResponseFactory Microsoft.Azure.Cosmos.CosmosResponseFactoryCore Endpoint https://localhost:8081/ AbsolutePath / AbsoluteUri https://localhost:8081/ LocalPath / Authority localhost:8081 HostNameType Dns IsDefaultPort False IsFile False IsLoopback True PathAndQuery / Segments [ / ] IsUnc False Host localhost Port 8081 Query Fragment Scheme https OriginalString https://localhost:8081 DnsSafeHost localhost IdnHost localhost IsAbsoluteUri True UserEscaped False UserInfo Headers [ x-ms-activity-id, x-ms-last-state-change-utc, etag, x-ms-resource-quota, x-ms-resource-usage, x-ms-schemaversion, lsn, x-ms-request-charge, x-ms-quorum-acked-lsn, x-ms-current-write-quorum, x-ms-current-replica-set-size, x-ms-xp-role, x-ms-global-Committed-lsn, x-ms-number-of-read-regions, x-ms-item-lsn, x-ms-transport-request-id, x-ms-cosmos-llsn, x-ms-cosmos-quorum-acked-llsn, x-ms-cosmos-item-llsn, x-ms-session-token ... (more) ] ContinuationToken <null> RequestCharge 2 ActivityId 0c54ce6a-51a7-4207-813a-a132ccc96627 ETag "00000000-0000-0000-1118-fad2e52a01dc" ContentType <null> Session 0:-1#31 ContentLength <null> Location <null> (values) index value 0 x-ms-activity-id 1 x-ms-last-state-change-utc 2 etag 3 x-ms-resource-quota 4 x-ms-resource-usage 5 x-ms-schemaversion 6 lsn 7 x-ms-request-charge 8 x-ms-quorum-acked-lsn 9 x-ms-current-write-quorum 10 x-ms-current-replica-set-size 11 x-ms-xp-role 12 x-ms-global-Committed-lsn 13 x-ms-number-of-read-regions 14 x-ms-item-lsn 15 x-ms-transport-request-id 16 x-ms-cosmos-llsn 17 x-ms-cosmos-quorum-acked-llsn 18 x-ms-cosmos-item-llsn 19 x-ms-session-token ... (more) Resource Microsoft.Azure.Cosmos.DatabaseProperties Id Live ETag "00000000-0000-0000-1118-fad2e52a01dc" LastModified 2025-08-19 14:53:09Z SelfLink dbs/iMI6AA==/ StatusCode OK Diagnostics {"Summary":{"DirectCalls":{"(200, 0)":2}},"name":"CreateDatabaseIfNotExistsAsync","start datetime":"2025-10-01T15:26:22.700Z","duration in milliseconds":17.5278,"data":{"Client Configuration":{"Client Created Time Utc":"2025-10-01T15:26:02.9390593Z","MachineId":"hashedMachineName:854b5717-0c71-e584-... Value Microsoft.Azure.Cosmos.Tracing.Trace Name CreateDatabaseIfNotExistsAsync Id f25ae56d-0a20-4e03-a00d-983677767af8 StartTime 2025-10-01 15:26:22Z Duration 00:00:00.0175278 Level Info Component Transport Summary Microsoft.Azure.Cosmos.Tracing.TraceSummary RegionsContacted index value 0 (South Central US, https://127.0.0.1:8081/) Item1 South Cent

In [10]:
var livedb = client.GetDatabase("Live");
var products = await livedb.CreateContainerIfNotExistsAsync("Products","/categoryid", 1000);
products.Display();

Microsoft.Azure.Cosmos.ContainerResponse Container Microsoft.Azure.Cosmos.ContainerInlineCore Id Products Database Microsoft.Azure.Cosmos.DatabaseInlineCore Id Live Client Microsoft.Azure.Cosmos.CosmosClient ClientOptions Microsoft.Azure.Cosmos.CosmosClientOptions ApplicationName <null> ApplicationRegion <null> ApplicationPreferredRegions <null> AccountInitializationCustomEndpoints <null> GatewayModeMaxConnectionLimit 50 RequestTimeout 00:00:06 TokenCredentialBackgroundRefreshInterval <null> CustomHandlers (empty) ConnectionMode Direct ConsistencyLevel <null> PriorityLevel <null> MaxRetryAttemptsOnRateLimitedRequests <null> MaxRetryWaitTimeOnRateLimitedRequests <null> EnableContentResponseOnWrite <null> UseSystemTextJsonSerializerWithOptions <null> IdleTcpConnectionTimeout <null> OpenTcpConnectionTimeout <null> MaxRequestsPerTcpConnection <null> MaxTcpConnectionsPerEndpoint <null> PortReuseMode <null> WebProxy <null> SerializerOptions <null> Serializer Microsoft.Azure.Cosmos.CosmosJsonSerializerWrapper LimitToEndpoint False AllowBulkExecution False EnableTcpConnectionEndpointRediscovery True HttpClientFactory <null> AvailabilityStrategy <null> ServerCertificateCustomValidationCallback <null> CosmosClientTelemetryOptions Microsoft.Azure.Cosmos.CosmosClientTelemetryOptions DisableSendingMetricsToService True DisableDistributedTracing True CosmosThresholdOptions Microsoft.Azure.Cosmos.CosmosThresholdOptions QueryTextMode None FaultInjector <null> ResponseFactory Microsoft.Azure.Cosmos.CosmosResponseFactoryCore Endpoint https://localhost:8081/ AbsolutePath / AbsoluteUri https://localhost:8081/ LocalPath / Authority localhost:8081 HostNameType Dns IsDefaultPort False IsFile False IsLoopback True PathAndQuery / Segments [ / ] IsUnc False Host localhost Port 8081 Query Fragment Scheme https OriginalString https://localhost:8081 DnsSafeHost localhost IdnHost localhost IsAbsoluteUri True UserEscaped False UserInfo LinkUri dbs/Live/colls/Products ClientContext Microsoft.Azure.Cosmos.ClientContextCore BatchExecutor <null> Conflicts Microsoft.Azure.Cosmos.ConflictsInlineCore Scripts Microsoft.Azure.Cosmos.Scripts.ScriptsInlineCore Headers [ Cache-Control, Pragma, Transfer-Encoding, Server, Access-Control-Allow-Origin, Access-Control-Allow-Credentials, x-ms-mtls-status, x-ms-activity-id, x-ms-last-state-change-utc, ETag, x-ms-schemaversion, collection-partition-index, collection-service-index, lsn, x-ms-request-charge, x-ms-content-path, x-ms-quorum-acked-lsn, x-ms-current-write-quorum, x-ms-current-replica-set-size, x-ms-documentdb-partitionkeyrangeid ... (more) ] ContinuationToken <null> RequestCharge 2 ActivityId 5dc88278-5c1a-4468-b6e3-b561a92be04d ETag "00000000-0000-0000-111b-0a9a766b01dc" ContentType application/json Session 0:-1#13 ContentLength <null> Location <null> (values) index value 0 Cache-Control 1 Pragma 2 Transfer-Encoding 3 Server 4 Access-Control-Allow-Origin 5 Access-Control-Allow-Credentials 6 x-ms-mtls-status 7 x-ms-activity-id 8 x-ms-last-state-change-utc 9 ETag 10 x-ms-schemaversion 11 collection-partition-index 12 collection-service-index 13 lsn 14 x-ms-request-charge 15 x-ms-content-path 16 x-ms-quorum-acked-lsn 17 x-ms-current-write-quorum 18 x-ms-current-replica-set-size 19 x-ms-documentdb-partitionkeyrangeid ... (more) Resource Microsoft.Azure.Cosmos.ContainerProperties PartitionKeyDefinitionVersion <null> ConflictResolutionPolicy Microsoft.Azure.Cosmos.ConflictResolutionPolicy Mode LastWriterWins ResolutionPath /_ts ResolutionProcedure Id Products UniqueKeyPolicy Microsoft.Azure.Cosmos.UniqueKeyPolicy UniqueKeys (empty) ETag "00000000-0000-0000-111b-0a9a766b01dc" LastModified 2025-08-19 15:07:55Z ClientEncryptionPolicy <null> IndexingPolicy Microsoft.Azure.Cosmos.IndexingPolicy Automatic True IndexingMode Consistent IncludedPaths index value 0 Microsoft.Azure.Cosmos.IncludedPath Path /* ExcludedPaths index value 0 Microsoft.Azure.Cosmos.ExcludedPath Path /"_etag"/? CompositeIndexes (empty) SpatialIndexes (emp

Rated
Auto Scale
Serverless

In [11]:
var containerprops = new ContainerProperties
{
    Id = "Products",
    PartitionKeyPath = "/categoryid"    
};

var throughputprops =  ThroughputProperties.CreateAutoscaleThroughput(4000);
var products = await livedb.CreateContainerIfNotExistsAsync(containerprops, throughputprops);
products.Display();


Microsoft.Azure.Cosmos.ContainerResponse Container Microsoft.Azure.Cosmos.ContainerInlineCore Id Products Database Microsoft.Azure.Cosmos.DatabaseInlineCore Id Live Client Microsoft.Azure.Cosmos.CosmosClient ClientOptions Microsoft.Azure.Cosmos.CosmosClientOptions ApplicationName <null> ApplicationRegion <null> ApplicationPreferredRegions <null> AccountInitializationCustomEndpoints <null> GatewayModeMaxConnectionLimit 50 RequestTimeout 00:00:06 TokenCredentialBackgroundRefreshInterval <null> CustomHandlers (empty) ConnectionMode Direct ConsistencyLevel <null> PriorityLevel <null> MaxRetryAttemptsOnRateLimitedRequests <null> MaxRetryWaitTimeOnRateLimitedRequests <null> EnableContentResponseOnWrite <null> UseSystemTextJsonSerializerWithOptions <null> IdleTcpConnectionTimeout <null> OpenTcpConnectionTimeout <null> MaxRequestsPerTcpConnection <null> MaxTcpConnectionsPerEndpoint <null> PortReuseMode <null> WebProxy <null> SerializerOptions <null> Serializer Microsoft.Azure.Cosmos.CosmosJsonSerializerWrapper LimitToEndpoint False AllowBulkExecution False EnableTcpConnectionEndpointRediscovery True HttpClientFactory <null> AvailabilityStrategy <null> ServerCertificateCustomValidationCallback <null> CosmosClientTelemetryOptions Microsoft.Azure.Cosmos.CosmosClientTelemetryOptions DisableSendingMetricsToService True DisableDistributedTracing True CosmosThresholdOptions Microsoft.Azure.Cosmos.CosmosThresholdOptions QueryTextMode None FaultInjector <null> ResponseFactory Microsoft.Azure.Cosmos.CosmosResponseFactoryCore Endpoint https://localhost:8081/ AbsolutePath / AbsoluteUri https://localhost:8081/ LocalPath / Authority localhost:8081 HostNameType Dns IsDefaultPort False IsFile False IsLoopback True PathAndQuery / Segments [ / ] IsUnc False Host localhost Port 8081 Query Fragment Scheme https OriginalString https://localhost:8081 DnsSafeHost localhost IdnHost localhost IsAbsoluteUri True UserEscaped False UserInfo LinkUri dbs/Live/colls/Products ClientContext Microsoft.Azure.Cosmos.ClientContextCore BatchExecutor <null> Conflicts Microsoft.Azure.Cosmos.ConflictsInlineCore Scripts Microsoft.Azure.Cosmos.Scripts.ScriptsInlineCore Headers [ Cache-Control, Pragma, Transfer-Encoding, Server, Access-Control-Allow-Origin, Access-Control-Allow-Credentials, x-ms-mtls-status, x-ms-activity-id, x-ms-last-state-change-utc, ETag, x-ms-schemaversion, collection-partition-index, collection-service-index, lsn, x-ms-request-charge, x-ms-content-path, x-ms-quorum-acked-lsn, x-ms-current-write-quorum, x-ms-current-replica-set-size, x-ms-documentdb-partitionkeyrangeid ... (more) ] ContinuationToken <null> RequestCharge 2 ActivityId be72f5ef-1840-462b-bca2-c302883005c2 ETag "00000000-0000-0000-111b-0a9a766b01dc" ContentType application/json Session 0:-1#13 ContentLength <null> Location <null> (values) index value 0 Cache-Control 1 Pragma 2 Transfer-Encoding 3 Server 4 Access-Control-Allow-Origin 5 Access-Control-Allow-Credentials 6 x-ms-mtls-status 7 x-ms-activity-id 8 x-ms-last-state-change-utc 9 ETag 10 x-ms-schemaversion 11 collection-partition-index 12 collection-service-index 13 lsn 14 x-ms-request-charge 15 x-ms-content-path 16 x-ms-quorum-acked-lsn 17 x-ms-current-write-quorum 18 x-ms-current-replica-set-size 19 x-ms-documentdb-partitionkeyrangeid ... (more) Resource Microsoft.Azure.Cosmos.ContainerProperties PartitionKeyDefinitionVersion <null> ConflictResolutionPolicy Microsoft.Azure.Cosmos.ConflictResolutionPolicy Mode LastWriterWins ResolutionPath /_ts ResolutionProcedure Id Products UniqueKeyPolicy Microsoft.Azure.Cosmos.UniqueKeyPolicy UniqueKeys (empty) ETag "00000000-0000-0000-111b-0a9a766b01dc" LastModified 2025-08-19 15:07:55Z ClientEncryptionPolicy <null> IndexingPolicy Microsoft.Azure.Cosmos.IndexingPolicy Automatic True IndexingMode Consistent IncludedPaths index value 0 Microsoft.Azure.Cosmos.IncludedPath Path /* ExcludedPaths index value 0 Microsoft.Azure.Cosmos.ExcludedPath Path /"_etag"/? CompositeIndexes (empty) SpatialIndexes (emp